Read the csv file from local directory

In [ ]:
filename = "data/housing.csv"
data = []
f = open(filename,'r',encoding="utf8")
df = f.readlines()
for line in df:
    line1=line.split(',')
    data.append(line1)
header = data[0]
values = data[1:]

Create a database connection and write helper functions for it.

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql, drop_table_name=None):
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()
    return rows

def insert_into_tables(conn, sql_insert_statement, values):
    sql = """ INSERT INTO EMPLOYEES VALUES(?, ?, ?) """
    cur = conn.cursor()
    cur.executemany(sql_insert_statement, values)
    return cur.lastrowid

conn = create_connection("database/USRent.db")

Create empty tables

In [ ]:
housing_table = "CREATE TABLE HousingList (listing_id INTEGER NOT NULL PRIMARY KEY REFERENCES MetaData(listing_id) , price INTEGER, sqfeet INTEGER, beds INTEGER, baths INTEGER, cats_allowed INTEGER, dogs_allowed INTEGER, smoking_allowed INTEGER, wheelchair_access INTEGER, electric_vehicle_charge INTEGER, comes_furnished INTEGER, location_id TEXT NOT NULL REFERENCES Location(location_id), laundry_id TEXT NOT NULL REFERENCES Laundry(laundry_id),housing_type_id TEXT NOT NULL REFERENCES HousingType(housing_type_id), parking_id TEXT NOT NULL REFERENCES Parking(parking_id))"

metadata_table = "CREATE TABLE MetaData (listing_id INTEGER NOT NULL PRIMARY KEY, url TEXT, image_url TEXT)"

location_table = "CREATE TABLE Location (location_id INTEGER NOT NULL PRIMARY KEY, region TEXT, state TEXT)"

laundry_table = "CREATE TABLE Laundry (laundry_id INTEGER NOT NULL PRIMARY KEY, option_name TEXT)"

housingtype_table = "CREATE TABLE HousingType (housing_type_id INTEGER NOT NULL PRIMARY KEY, housing_type TEXT)"

parking_table = "CREATE TABLE Parking (parking_id INTEGER NOT NULL PRIMARY KEY, parking_type TEXT)"

with conn:
    create_table(conn, housing_table, "HousingList")
    create_table(conn, metadata_table, "MetaData")
    create_table(conn, location_table, "Location")
    create_table(conn, laundry_table, "Laundry")
    create_table(conn, housingtype_table, "HousingType")
    create_table(conn, parking_table, "Parking")


Insert values into the laundry table

In [ ]:
options = []
for i in values:
    try:
        options.append(i[15])
    except IndexError:
        continue
options = list(set(options))
insert_laundry_table = "Insert into Laundry(option_name) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_laundry_table,[(option,) for option in options])

sql_statement = "select * from Laundry;"
execute_sql_statement(sql_statement, conn)

Insert values into the housing type table

In [ ]:
type = []
for i in values:
    try:
        type.append(i[5])
    except IndexError:
        continue
type = list(set(type))
insert_type_table = "Insert into HousingType(housing_type) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_type_table,[(j,) for j in type])

sql_statement = "select * from HousingType;"
execute_sql_statement(sql_statement, conn)

Insert values into the parking type table 

In [ ]:
parking = []
for i in values:

    try:
        parking.append(i[16])
    except IndexError:
        continue
parking = list(set(parking))
insert_parking_table = "Insert into Parking(parking_type) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_parking_table,[(p,) for p in parking])


sql_statement = "select * from Parking;"
execute_sql_statement(sql_statement, conn)


Insert values into the location table

In [ ]:
location = []
for i in values:
    try:
        a = i[20].strip()
        location.append(i[2]+'*'+a)
    except IndexError:
        continue

location = list(set(location))

st=[]
loc=[]
for i in location:
    a=i.split('*')
    st.append(a[1])
    loc.append(a[0])

insert_location_table = "Insert into Location(region,state) Values(?,?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_location_table,[(l,s) for l,s in zip(loc,st)])

#have to split and save region and state

sql_statement = "select * from Location;"
execute_sql_statement(sql_statement, conn)

Insert values into the metadata table

In [ ]:
url = []
img_url=[]
l_id = []
for i in values:
    try:
        url.append(i[1])
        img_url.append(i[17])
        l_id.append(i[0])
    except IndexError:
        continue
        
insert_metadata_table = "Insert into MetaData(listing_id,url,image_url) Values(?,?,?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_metadata_table,[(i,j,k) for i,j,k in zip(l_id,url,img_url)])

sql_statement = "select * from MetaData;"
execute_sql_statement(sql_statement, conn)

Helper function to map different parameter types to their associated id

In [ ]:
def get_type(name,db_name):
    sql_statement = f"select * from {db_name}"
    with conn:
        try:
            c = execute_sql_statement(sql_statement, conn)
            laundry_dict = {x[1]: x[0] for x in c}
            print(laundry_dict)
            return laundry_dict[name]
        except KeyError:
            return None


In [ ]:
def get_location(name):
    sql_statement = "select location_id, region||state from Location"
    with conn:
        try:
            c = execute_sql_statement(sql_statement, conn)
            location_dict = {x[1]: x[0] for x in c}
            print(location_dict)
            return location_dict[name]
        except KeyError:
            return None

Insert data into the main housing list table 

In [ ]:
input_tup = []
for i in values:
    try:
        l_id=i[0]
        price=i[4]
        sq_feet=i[6] 
        bed=i[7]
        bath=i[8]
        cats=i[9]
        dogs=i[10]
        smoking=i[11]
        wchair=i[12]
        e_charge=i[13]
        furnished=i[14]
        location = f"{i[2]}{i[20]}".replace('\n',"")
        loc = get_location(location)
        h_type = get_type(i[5],"HousingType")
        park_type = get_type(i[16],"Parking")
        ldry = get_type(i[15],"Laundry")
        insert_HousingList_table = "Insert into HousingList(listing_id,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access, electric_vehicle_charge, comes_furnished, location_id, laundry_id,housing_type_id, parking_id) Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);"
        input_tup.append((l_id,price,sq_feet,bed,bath,cats,dogs,smoking,wchair,e_charge,furnished,loc,ldry,h_type,park_type))

    except Error as e:
        print(e)
        continue
        
with conn:
    cur = conn.cursor()
    cur.executemany(insert_HousingList_table,input_tup)

sql_statement = "select count(*) from HousingList;"
execute_sql_statement(sql_statement, conn)

In [ ]:
sql_statement = "select count(*) from HousingList;"
execute_sql_statement(sql_statement, conn)